# Import packages

In [1]:
import requests
import pandas as pd
import logging
import json
from collections import defaultdict
import time
import re
import unicodedata
logging.basicConfig(level=logging.INFO)

# Read ORCAS

In [2]:
df = pd.read_csv('/content/drive/My Drive/Project 3/orcas.tsv', sep='\t', header=None, encoding='utf-8')

In [3]:
df.rename(columns={0:'qid', 1:'query', 2:'did', 3:'url'}, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
# df['id'] = df.index

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [4]:
df['url'] = df['url'].apply(lambda x: unicodedata.normalize("NFC", x))
df['query'] = df['query'].apply(lambda x: unicodedata.normalize("NFC", x))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18823526 entries, 0 to 18823525
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   qid     int64 
 1   query   object
 2   did     object
 3   url     object
dtypes: int64(1), object(3)
memory usage: 574.4+ MB


In [ ]:
# df.to_csv("/content/drive/My Drive/Project 3/orcas.csv", encoding='utf-8', index=None)

# Load search results

In [6]:
with open("/content/drive/My Drive/Project 3/linksOfMovies.txt") as f:
  linksOfMovies = json.load(f)

In [18]:
links = linksOfMovies['Harry Potter and the Goblet of Fire']
for i in range(len(links) - 1):
  temp = links[i].split("://")[1]
  for j in range(i+1, len(links)):
    if links[j].find(temp) != -1:
      print(links[i])
      print(links[j])

https://en.wikipedia.org/wiki/Harry_Potter_and_the_Goblet_of_Fire
https://en.wikipedia.org/wiki/Harry_Potter_and_the_Goblet_of_Fire_(film)


In [19]:
df[df.url == 'https://en.wikipedia.org/wiki/Harry_Potter_and_the_Goblet_of_Fire_(film)']

,qid,query,did,url
202015,10335859,4th harry potter film,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517788,8655240,goblet of fire,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517791,10555984,goblet of fire cast,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517792,12631112,goblet of fire film,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517794,6712870,goblet of fire movie,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517797,10625327,goblet of fire wiki,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6896275,9628598,harry potter 2005,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6896294,6128243,harry potter 4,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6896299,6903790,harry potter 4 full movie,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6896302,10683817,harry potter 4 movie,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...


In [21]:
df[df.url.str.contains('en.wikipedia.org/wiki/Harry_Potter_and_the_Goblet_of_Fire')]

,qid,query,did,url
202013,2068014,4th harry potter,D2131152,https://en.wikipedia.org/wiki/Harry_Potter_and...
202014,6763897,4th harry potter book,D2131152,https://en.wikipedia.org/wiki/Harry_Potter_and...
202015,10335859,4th harry potter film,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6040651,6385308,fourth harry potter,D2131152,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517788,8655240,goblet of fire,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517790,8655240,goblet of fire,D2131152,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517791,10555984,goblet of fire cast,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517792,12631112,goblet of fire film,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517793,6311649,goblet of fire harry potter,D2131152,https://en.wikipedia.org/wiki/Harry_Potter_and...
6517794,6712870,goblet of fire movie,D2131153,https://en.wikipedia.org/wiki/Harry_Potter_and...


In [7]:
with open("/content/drive/My Drive/Project 3/movies.txt") as f:
  movies = f.read()

movies = movies.split("\n")[:-1]

In [8]:
movies[90:]

['Thor: The Dark World',
 'Three Billboards Outside Ebbing, Missouri',
 'Transformers: Age of Extinction',
 'Transformers: Dark of the Moon',
 'Valerian and the City of a Thousand Planets',
 'World War Z',
 'Wreck-It Ralph',
 'X-Men Origins: Wolverine',
 'X-Men: Apocalypse',
 'X-Men: Days of Future Past',
 'X-Men: The Last Stand']

In [9]:
print("Total movies: ",len(movies))

Total movies:  101


# Find synonym candidates for each movies

In [10]:
# danh sách chứa candidate của từng movie 
candidatesOfMovies = defaultdict(list)
for movie in movies[0:25]:
  print("Movie: ", movie)
  t1 = time.time()
  candidatesOfMovies[movie] = defaultdict(list)
  for link in linksOfMovies[movie]:
    tempLink = link.split("://")[1]
    print("   TempLink: ", tempLink)
    df_candidates = df[df.url.str.contains(tempLink, regex=False)]
    # nếu tìm thấy thì lưu lại link, did, qid, query
    print("     Length of df_candidates: ", len(df_candidates))
    if len(df_candidates) > 0:
      for id, row in df_candidates.iterrows():
        candidatesOfMovies[movie][row['qid']].append(row['did'])
  print("Total candidates of this movie (co the trung): ", len(candidatesOfMovies[movie]))
  print("Time to find = ", time.time() - t1)

Movie:  Alice Through the Looking Glass
   TempLink:  www.imdb.com/title/tt2567026/
     Length of df_candidates:  25
   TempLink:  en.wikipedia.org/wiki/Alice_Through_the_Looking_Glass_(2016_film)
     Length of df_candidates:  0
   TempLink:  movies.disney.com/alice-through-the-looking-glass
     Length of df_candidates:  2
   TempLink:  en.wikipedia.org/wiki/Through_the_Looking-Glass
     Length of df_candidates:  29
   TempLink:  www.rottentomatoes.com/m/alice_in_wonderland_through_the_looking_glass
     Length of df_candidates:  0
   TempLink:  disney.fandom.com/wiki/Alice_Through_the_Looking_Glass
     Length of df_candidates:  0
   TempLink:  www.imdb.com/title/tt2567026/fullcredits
     Length of df_candidates:  0
   TempLink:  www.sparknotes.com/lit/through-the-looking-glass/summary/
     Length of df_candidates:  0
   TempLink:  www.123movie.lc/mov/alice-through-the-looking-glass-2016/
     Length of df_candidates:  0
   TempLink:  bookroo.com/quotes/through-the-looking-glass

In [ ]:
candidatesOfMovies

In [15]:
for movie in candidatesOfMovies:
  for qid in candidatesOfMovies[movie]:
    candidatesOfMovies[movie][qid] = list(set(candidatesOfMovies[movie][qid]))

In [ ]:
candidatesOfMovies

# Remove movies which dont have any candidates.

In [22]:
for movie in candidatesOfMovies:
  if len(candidatesOfMovies[movie]) == 0:
    print("Movie {} doesn't have any candidates.".format(movie))
    # del candidatesOfMovies[movie]
print("Total movie: ", len(candidatesOfMovies.keys()))

Total movie:  25


In [23]:
len(candidatesOfMovies)

25

# Save candidates of Movies

In [24]:
with open("/content/drive/My Drive/Project 3/CandidatesOfMovies(4).json", 'w',
          encoding='utf-8') as f:
  json.dump(candidatesOfMovies, f)

# Tong hop

Chay cung luc 4 tien trinh de tang toc do tim kiem canidates. Tong hop cac ket qua

In [26]:
candidates = dict()
for i in range(1, 5):
  with open("/content/drive/My Drive/Project 3/CandidatesOfMovies("+str(i)+").json",
          encoding='utf-8') as f:
    temp = json.load(f)
    candidates.update(temp)

In [28]:
len(candidates.keys())

101

In [29]:
with open("/content/drive/My Drive/Project 3/CandidatesOfMovies.json", 'w',
          encoding='utf-8') as f:
  json.dump(candidates, f)

In [ ]:
listCandidates = []
# did2url1 = {}
for movie in candidates:
  print(movie)
  for qid in candidates[movie]:
    query = df[df['qid'] == int(qid)]['query'].unique().item()
    print(query)
    did = ",".join(candidates[movie][qid])
    url = []
    for did in candidates[movie][qid]:
      if did not in did2url:
        did2url[did] = df[df['did'] == did]['url'].unique().item()
      url.append(did2url[did])
    # print(url)
    url = ",".join(url)
    listCandidates.append([movie, qid, query, did, url])

In [60]:
df_candidates = pd.DataFrame(listCandidates, columns=['movie', 'qid', 'query', 'did', 'url'])

In [61]:
df_candidates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7717 entries, 0 to 7716
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   movie   7717 non-null   object
 1   qid     7717 non-null   object
 2   query   7717 non-null   object
 3   did     7717 non-null   object
 4   url     7717 non-null   object
dtypes: object(5)
memory usage: 301.6+ KB


In [76]:
df_candidates.head()

,movie,qid,query,did,url,check
0,Harry Potter and the Order of the Phoenix,10484102,fifth harry potter movie,D2386020,http://www.imdb.com/title/tt0373889/,
1,Harry Potter and the Order of the Phoenix,8174032,harry potter,D2386020,http://www.imdb.com/title/tt0373889/,n
2,Harry Potter and the Order of the Phoenix,10083356,harry potter 5,D2091109,https://www.goodreads.com/book/show/2.Harry_Po...,
3,Harry Potter and the Order of the Phoenix,9596611,harry potter and the order of phoenix,D2091109,https://en.wikipedia.org/wiki/Harry_Potter_and...,
4,Harry Potter and the Order of the Phoenix,4046120,harry potter and the order of phoenix full movie,D2901598,"http://www.imdb.com/title/tt0373889/,https://w...",


In [63]:
count = df_candidates['qid'].value_counts()

In [64]:
count[count > 1].index

Index(['5748076', '3219419', '2931776', '2217938', '7714491', '4087032',
       '9372758', '11678771', '6175331', '9542393',
       ...
       '5463261', '5501019', '11779287', '6191735', '4718477', '8389809',
       '4711812', '2619581', '2730713', '4598801'],
      dtype='object', length=494)

In [65]:
df_candidates['check'] = ''

In [70]:
index = df_candidates[df_candidates['qid'].isin(count[count > 1].index)].index

In [75]:
df_candidates.loc[index, 'check'] = 'n'

,movie,qid,query,did,url,check
1,Harry Potter and the Order of the Phoenix,8174032,harry potter,D2386020,http://www.imdb.com/title/tt0373889/,n
35,Harry Potter and the Order of the Phoenix,5358239,harry potter and the goblet of fire,D2091109,https://en.wikipedia.org/wiki/Harry_Potter_and...,n
79,Harry Potter and the Order of the Phoenix,4122326,harry potter full movie,D2901598,https://www.rottentomatoes.com/m/harry_potter_...,n
83,Hotel Transylvania,5392248,cast of hotel transylvania,D1284778,https://en.wikipedia.org/wiki/Hotel_Transylvania,n
84,Hotel Transylvania,4370206,cast of hotel transylvania 2,D1048457,https://en.wikipedia.org/wiki/Hotel_Transylvan...,n
...,...,...,...,...,...,...
7635,Guardians of the Galaxy Vol. 2,6887024,the guardian of the galaxy 2,D948069,https://en.wikipedia.org/wiki/Guardians_of_the...,n
7636,Guardians of the Galaxy Vol. 2,8635181,vol 2,D948069,https://en.wikipedia.org/wiki/Guardians_of_the...,n
7667,Harry Potter and the Goblet of Fire,8174032,harry potter,D2131150,https://en.wikipedia.org/wiki/Harry_Potter_and...,n
7673,Harry Potter and the Goblet of Fire,5358239,harry potter and the goblet of fire,D2131150,https://en.wikipedia.org/wiki/Harry_Potter_and...,n


In [77]:
df_candidates.to_csv("/content/drive/My Drive/Project 3/candidatesOfMovies.csv", index=False,
                     encoding='utf-8')

# Tim docid

In [78]:
# get all query ID.
qids = df_candidates.qid.unique()

In [79]:
df_temp = df[df.qid.isin(qids)]

In [80]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15799 entries, 1084 to 18820648
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     15799 non-null  int64 
 1   query   15799 non-null  object
 2   did     15799 non-null  object
 3   url     15799 non-null  object
dtypes: int64(1), object(3)
memory usage: 617.1+ KB


In [83]:
dids = df_temp.did.unique().tolist()

In [85]:
# save
with open('/content/drive/My Drive/Project 3/documentID.txt', 'w', encoding='utf-8') as f:
  for did in dids:
    f.write(did + "\n")